# [Graph Search, Shortest Paths, and Data Structures - Week4](https://www.coursera.org/learn/algorithms-graphs-data-structures/home/week/4)

- Hashing
- Universal Hashing
- Bloom Filters

SUGGESTED READINGS FOR WEEK 4: Algorithms Illuminated (Part 2), Chapter 12.

## XIV. HASHING: THE BASICS (Week 4)

### [Hash Tables: Operations and Applications](https://www.coursera.org/learn/algorithms-graphs-data-structures/lecture/b2Uee/hash-tables-operations-and-applications)

- hash table 能 support 的 operations 不多，但是卻做得很好。
- 可以把 hash table 當成 array
    - array 支援 random access，即我要拿第 $i$ 個 element
        - > Q: 但是 hash table??
- 但是 array 有些限制，例如我想存很多人的電話：
    - key 必須是「某個範圍內的」「integer」
    - 人名有太多種了，一次要 array 能容納所有可能的人名很困難
- 我們希望 array 有 reasonable size

#### Hash Table: Supported Operations

![](https://i.imgur.com/6i98lKm.png)
- 如果有 task 需要比較 (例如 maximum)，就不適合 hash table
- 雖然說 insert delete lookup 都可以在 constant time $O(1)$ 內達成，但是
    1. 很難 implement 得好，如果 implement 得不好那就不會是 constant time。
    2. worst case 其實並不是 constant time，只有 non-pathological(非病態?) dataset

#### Application: De-Duplication

![](https://i.imgur.com/rWylekk.png)
- **"stream"** 可以看下面幾個例子：
    1. 你要 scan 一個超大的 file，所以你 scan line by line。
    2. 會隨著時間一直收到 data，例如 router 會一直收到 data packet

#### Application: The 2-Sum Problem

![](https://i.imgur.com/Of2te2K.png)
- 最 naive 的做法就是用兩層 loop 跑
- 好一點的做法是：
    - 可以先 sort (想要比 $\theta(n^2)$ 好，可以先 sort ($\theta(n\log n)$) 看看能不能幫助 two-sum task。)
        - > **meta-knowledge: 想要比 某個複雜度 好，總是可以先做一些 (比該複雜度快的) pre-processing 看看能不能幫助該 task**
    - 然後跑每個數字 $x\in A$ 找 sorted array 裡面有沒有 $t-x$ (binary search 只需要 $\theta(\log n)$)
    - 這樣就只需要 $\theta(n\log n)$
- 再仔細想想，我們需要的其實是：對於每個數字 $x\in A$ 找 $A$ 裡面有沒有 $t-x$，**也就是我們只需要 lookup !!! 該用 hash table 了**
    - 所以只需要先 preprocess 成 hash table。(insert $n$ 次，$O(n)$)
    - 再對每個 $x$ 去 search $t-x$ 即可。(search $n$ 次，$O(n)$)
    - 所以總共只要 $O(n)$。

#### Further Immediate Applications

![](https://i.imgur.com/2horu0X.png)
- symbol tables: 需要檢查 variables 是否已經 defined。
- 把 blacklist 的 IP address 給 block 掉
- search algorithms，這邊說的是棋類遊戲的 configurations (配置；構造) search
    - 之前教的 BFS、DFS 都假設我們的 graph 可以直接存在 main memory
    - 但是這種棋類遊戲的 graph (vertices 是 configurations，edges 是 legal moves) 太大了，根本沒辦法記錄下來，因此沒辦法套 BFS 或 DFS。
    - 而我們還是想 explore 這個 graph，但又不想再遇到已經 explored 的，因為不想做 redundant work。
    - 這時候我們需要做的就是 lookup，因此 hash table 又派上用場啦~

### [Hash Tables: Implementation Details, Part I](https://www.coursera.org/learn/algorithms-graphs-data-structures/lecture/Ob0K7/hash-tables-implementation-details-part-i)

#### Hash Table: Supported Operations (Review)
- 回顧一下啊

#### High-Level Idea

![](https://i.imgur.com/fReK8Ap.png)
- universe 的 size 通常 超 級 大，例如所有可能的 names 大約有 $26^{30}$ 種
- 這邊的 reasonable size 可以是 thousands 到 millions，都比 universe 的 size 小多了
- 注意 evolving set $S$ 是可能變動的，不過我們先假設 $S$ 不會變動太多
- naive solution
    - array-based 佔用太多 space
    - list-based 花費太多 time

#### Quiz: Birthday Paradox

![](https://i.imgur.com/7pFCX8Y.png)
- 全都不 collide 的機率為 $\frac{365}{365}\times \frac{364}{365}\times \frac{363}{365}\times ...\times \frac{365-(n-1)}{365}=\frac{365\times ...\times (365-(n-1))}{365^n}$
- collide 的機率為 $1-\frac{365\times ...\times (365-(n-1))}{365^n}$
- 即使是 tiny dataset 而且 hash function 可以很 uniform 的分配到各個 bucket，仍然有很高機率會發生 collision。

In [1]:
def show_collide_prob(m):
    n = 365
    prod = 1
    for i in range(m):
        prod *= (n-i)/n
    prob = 1 - prod
    print('collide probability is %f for m = %d'%(prob, m))

show_collide_prob(23)
show_collide_prob(57)
show_collide_prob(367)


collide probability is 0.507297 for m = 23
collide probability is 0.990122 for m = 57
collide probability is 1.000000 for m = 367


#### Resolving Collisions

![](https://i.imgur.com/zTSqot0.png)
- Solution #1: chaining
- Solution #2: open addressing
    - 如果發生 collision 在某個 bucket，就要找別的地方來安置 collided object。
    - 本來是 hash function $h(x)$，現在是 hash sequence $h_1(x),h_2(x),...$，例如：
        - linear probing: $h(x)$ 有人了就下一個 $h(x)+1$ ...
        - double hashing: 使用兩組 hash function 共同決定位置?
- 那麼 solution #1 和 #2 誰比較好? 各有優劣。
    - 若重視 space，就選 #2
    - 但是 #2 的 deletion 比較 tricky (難辦的@@)
- 接下來就要講「如何設計一個好的 hash function」

### [Hash Tables: Implementation Details, Part II](https://www.coursera.org/learn/algorithms-graphs-data-structures/lecture/1Y8t1/hash-tables-implementation-details-part-ii)

#### What Makes a Good Hash Function?

![](https://i.imgur.com/xd2qrnZ.png)
- $m$ objects, $n$ buckets
- insert $\theta(1)$ 因為直接 insert 在 bucket 的 **front** of linked list
    - > Q: 為何說 insert 是 $\theta(1)$ 呢? 不用先 search chain (list) 再決定要不要 insert 嗎?
- $\theta(\textrm{list length})$ for **Lookup**/Delete. [slide typo(?)]
    - 最好的情況 (hash function 很平均的把 keys 分配到各 buckets)：每個 bucket 的 **list 長度都是固定的 (還是說要小於 $n$ @@?)** (那長度就會是 $m/n$)，那麼 lookup 就會是 constant time $\theta(1)$
    - 最壞的情況 (hash function 把 keys 都分配到同個 bucket)：那麼就會有 bucket 的 list 長度為 $m$，複雜度直接變 $\theta(m)$
- **所以 hash function 的選擇非常重要！**
    - 希望可以將 keys 映射得非常分散，那 random 行不行呢?
    - 不行，因為這樣還是要 remember 每個 key 對應到的 random indices，那 indices 怎麼存? 又變回原本的問題了。所以不能直接 random。
    - 另外，因為我們想要 constant time look up，所以 hash function 的 evaluate (計算?) 也要 constant time。


#### Bad Hash Functions

![](https://i.imgur.com/5El9Ok7.png)
- Example #1
    - first 3 digits of $x$: terrible 因為：
        1. 可能你蒐集到的電話幾乎都集中在某些 area 因此有相同的 area code
        2. 電話前三碼並不是任何數字組合都是 legal 的，因此會有很多 empty bucket。
- Example #2
    - $x$ mod 1000 terrible 因為：
        - 奇數是 illegal memory address (因為一次至少存一個 byte?)，於是又會導致很多 empty bucket。
- 總之，很容易不小心就設計出很拉基的 hash function。
    - 導致許多 empty bucket 的 hash function 就是拉基。
    - > 看來 hash function 的好壞也跟 domain (data distribution) 有關?

#### Quick and Dirty Hash Functions

![](https://i.imgur.com/LZrPoQw.png)
- 一般可以把 hash function 分成兩階段討論：**hash code**, **compression function**[slide typo]
    - hash code 就是單純把 key 映射成 integer，例如把 string 利用 ASCII code 映射成整數
    - compression function 就是把整數再 map 到 array index。
- 如何選擇 $n$(# of buckets) 呢?
    - 如果 $n$ 和 「有意義的 digit (???)」有公因數的話，就很容易產生前一張 slide 的問題，因此最簡單的方法是選一個大小合適的質數。
    - 以電話號碼來說，當十進位表示時，會有特殊的意義，因此容易產生問題，所以不要 mod 接近 10 的某次方?
    - 以 memory address 來說，二進位表示，會有特殊意義，因此也容易產生問題，所以不要 mod 接近 2 的某次方?
        - > Q: 這邊的邏輯不太懂@@

## XV. UNIVERSAL HASHING (Week 4)

### [Pathological Data Sets and Universal Hashing Motivation](https://www.coursera.org/learn/algorithms-graphs-data-structures/lecture/1SQo3/pathological-data-sets-and-universal-hashing-motivation)

#### Hash Table: Supported Operations (Review)

![](https://i.imgur.com/1ZGKRBE.png)

#### Resolving Collisions (Review)

![](https://i.imgur.com/yoP1lLt.png)

#### The Load of a Hash Table
- Definition: the **load factor** of a hash table is
    - $\alpha:=\dfrac{\textrm{# of objects in hash table}}{\textrm{# of buckets of hash table}}$

#### Quiz: Load Factor

![](https://i.imgur.com/UYlPShj.png)

#### The Load of a Hash Table (con'd)

![](https://i.imgur.com/9MmQKQ9.png)
- 使用 open addressing 的話，甚至會希望 load factor $\alpha \ll 1$
- 想要 hash table 有好的 performance，我們就必須控制 load factor。
    - 怎麼控制? insert 跟 delete 不是我們能決定的啊！那就控制 # of buckets，例如，$\alpha$ 大於某個值就把 # of buckets 變兩倍，並調整 hash function。

#### Pathological Data Sets

![](https://i.imgur.com/MlNGF9Y.png)
- 理想：希望不論 data 長怎樣，hash function 都可以 spread data out evenly
    - **很可惜，不可能。**
    - 原因：當你決定好(即 fix 住) hash function，那麼利用鴿籠原理可以知道，因為 $|U|\gg n$，所以一定找得到 dataset 能打壞平衡。

#### Pathological Data in the Real World

![](https://i.imgur.com/h8wiVYn.png)
- 現實中，我們可以針對 hash function 做攻擊，利用 reverse engineering 來導致 bad performance。

#### Solutions

![](https://i.imgur.com/q3G0bmC.png)
- Sol 1. 比較偏向 practical solution (防止有人惡意地 construct pathological dataset)
- Sol 2. 比較偏向 mathematical solution?
    - 設計 a family of hash functions，然後每次 randomly choose one。
    - 這樣 average 來說，也可以降低 pathological dataset 的機率。(原理類似 quick sort 降低 worst case 機率)
    - 這樣其實也可以防止 reverse engineering。

#### Overview of Universal Hashing

![](https://i.imgur.com/YoJSfm1.png)
- > Q: 這幾段不是很懂，改天回來看@@

### [Universal Hashing: Definition and Example [Advanced - Optional]](https://www.coursera.org/learn/algorithms-graphs-data-structures/lecture/NA9cO/universal-hashing-definition-and-example-advanced-optional)

### [Universal Hashing: Analysis of Chaining [Advanced - Optional]](https://www.coursera.org/learn/algorithms-graphs-data-structures/lecture/YhHRN/universal-hashing-analysis-of-chaining-advanced-optional)

### [Hash Table Performance with Open Addressing [Advanced - Optional]](https://www.coursera.org/learn/algorithms-graphs-data-structures/lecture/3bPLu/hash-table-performance-with-open-addressing-advanced-optional)

## XVI. BLOOM FILTERS (Week 4)

### [Bloom Filters: The Basics](https://www.coursera.org/learn/algorithms-graphs-data-structures/lecture/riKfa/bloom-filters-the-basics)

#### Bloom Filters: Supported Operations

![](https://i.imgur.com/ZdgLbwi.png)
- 原來是因為 developed by Burton Bloom 所以叫 Bloom filters...
- No deletions 的部分，其實硬要 delete 也是可以，但是非常耗費時間。
- 會有 false positive，但不會有 false negative。

#### Bloom Filters: Applications

![](https://i.imgur.com/zxzguep.png)
- Bloom Filters 使用時機：很 care 花費了多少 space，不太 care 少數的 false positives。
- Original: early spellcheckers 拼寫檢查程式 (乾難怪有時候明明就輸入了正常的 word 卻給我畫紅線
- Canonical: list of forbidden passwords (which are too simple to guess)
- Modern: network routers
    - 有時候要 block 某些 IP
    - 或者記住某些 cache?


#### Bloom Filter: Under the Hood

![](https://i.imgur.com/mThgd31.png)
- $n$ bits 是整個 array 的 size，所以 $\frac{n}{|S|}$ 就是 data set $S$ 當中每個 object 的 # of bits。

### [Bloom Filters: Heuristic Analysis](https://www.coursera.org/learn/algorithms-graphs-data-structures/lecture/QSHNY/bloom-filters-heuristic-analysis)

#### Heuristic Analysis

![](https://i.imgur.com/lMs9Rdx.png)

#### Quiz: Probability of a Bit

![](https://i.imgur.com/DbgAJMy.png)

#### Heuristic Analysis (con'd)

![](https://i.imgur.com/778Ohs5.png)

#### Heuristic Analysis (con'd)

![](https://i.imgur.com/9QclOEC.png)